In [1]:
from collections import OrderedDict
import os
import glob
import numpy as np
import pandas as pd
import dask
from dask.delayed import delayed
from dask.distributed import Client, wait
import dask_cudf
import shutil

In [2]:
use_local_cuda_cluster = False
# Use below for a local-only CUDA cluster
if use_local_cuda_cluster:
    from dask_cuda import LocalCUDACluster
    cluster = LocalCUDACluster(ip='0.0.0.0')
    client = Client(cluster)

In [3]:
# Use below for a multi-host multi-GPU CUDA cluster started with start_dask.py.
if not use_local_cuda_cluster:
    scheduler_address = '10.200.11.12:8786'
    client = Client(address=scheduler_address)

In [4]:
client

Client Scheduler: tcp://10.200.11.12:8786 Dashboard: http://10.200.11.12:8787/status,Cluster Workers: 32 Cores: 160 Memory: 2.20 TB


In [5]:
%%time
# Restart all workers. This also clears GPU memory.
client.restart()

CPU times: user 20.1 ms, sys: 155 µs, total: 20.3 ms
Wall time: 18 s


Client Scheduler: tcp://10.200.11.12:8786 Dashboard: http://10.200.11.12:8787/status,Cluster Workers: 32 Cores: 160 Memory: 2.20 TB


In [6]:
# Below from mortgage/E2E.ipynb.
def gpu_load_performance_csv(performance_path, **kwargs):
    """ Loads performance data

    Returns
    -------
    GPU DataFrame
    """
    
    cols = [
        "loan_id", "monthly_reporting_period", "servicer", "interest_rate", "current_actual_upb",
        "loan_age", "remaining_months_to_legal_maturity", "adj_remaining_months_to_maturity",
        "maturity_date", "msa", "current_loan_delinquency_status", "mod_flag", "zero_balance_code",
        "zero_balance_effective_date", "last_paid_installment_date", "foreclosed_after",
        "disposition_date", "foreclosure_costs", "prop_preservation_and_repair_costs",
        "asset_recovery_costs", "misc_holding_expenses", "holding_taxes", "net_sale_proceeds",
        "credit_enhancement_proceeds", "repurchase_make_whole_proceeds", "other_foreclosure_proceeds",
        "non_interest_bearing_upb", "principal_forgiveness_upb", "repurchase_make_whole_proceeds_flag",
        "foreclosure_principal_write_off_amount", "servicing_activity_indicator"
    ]
    
    # Date dtype is not used because the Parquet reader gives an error when reading it.
    # Category dtype is not used. It is not clear if it works correctly in dask_cudf.
    dtypes = OrderedDict([
        ("loan_id", "int64"),
        ("monthly_reporting_period", "str"),
        ("servicer", "str"),
        ("interest_rate", "float64"),
        ("current_actual_upb", "float64"),
        ("loan_age", "float64"),
        ("remaining_months_to_legal_maturity", "float64"),
        ("adj_remaining_months_to_maturity", "float64"),
        ("maturity_date", "str"),
        ("msa", "float64"),
        ("current_loan_delinquency_status", "int32"),
        ("mod_flag", "str"),
        ("zero_balance_code", "str"),
        ("zero_balance_effective_date", "str"),
        ("last_paid_installment_date", "str"),
        ("foreclosed_after", "str"),
        ("disposition_date", "str"),
        ("foreclosure_costs", "float64"),
        ("prop_preservation_and_repair_costs", "float64"),
        ("asset_recovery_costs", "float64"),
        ("misc_holding_expenses", "float64"),
        ("holding_taxes", "float64"),
        ("net_sale_proceeds", "float64"),
        ("credit_enhancement_proceeds", "float64"),
        ("repurchase_make_whole_proceeds", "float64"),
        ("other_foreclosure_proceeds", "float64"),
        ("non_interest_bearing_upb", "float64"),
        ("principal_forgiveness_upb", "float64"),
        ("repurchase_make_whole_proceeds_flag", "str"),
        ("foreclosure_principal_write_off_amount", "float64"),
        ("servicing_activity_indicator", "str")
    ])

    return dask_cudf.read_csv(performance_path, names=cols, delimiter='|', dtype=list(dtypes.values()))

In [7]:
# Identify list of files to load.
data_dir = '/mnt/isilon1/data/mortgage'
perf_file = []
perf_file += glob.glob(data_dir + '/perf/Performance_*')
#perf_file += glob.glob(data_dir + '/perf/Performance_2016*.txt')
len(perf_file)

112

In [8]:
%%time
perf_ddf = gpu_load_performance_csv(perf_file)

CPU times: user 1.05 s, sys: 818 ms, total: 1.87 s
Wall time: 2.08 s


In [9]:
perf_ddf

,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,current_loan_delinquency_status,mod_flag,zero_balance_code,zero_balance_effective_date,last_paid_installment_date,foreclosed_after,disposition_date,foreclosure_costs,prop_preservation_and_repair_costs,asset_recovery_costs,misc_holding_expenses,holding_taxes,net_sale_proceeds,credit_enhancement_proceeds,repurchase_make_whole_proceeds,other_foreclosure_proceeds,non_interest_bearing_upb,principal_forgiveness_upb,repurchase_make_whole_proceeds_flag,foreclosure_principal_write_off_amount,servicing_activity_indicator
npartitions=823,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,float64,float64,float64,float64,float64,object,float64,int32,object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [10]:
%%time
perf_ddf.head()

CPU times: user 209 ms, sys: 9.53 ms, total: 219 ms
Wall time: 2.47 s


,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,holding_taxes,net_sale_proceeds,credit_enhancement_proceeds,repurchase_make_whole_proceeds,other_foreclosure_proceeds,non_interest_bearing_upb,principal_forgiveness_upb,repurchase_make_whole_proceeds_flag,foreclosure_principal_write_off_amount,servicing_activity_indicator
0,548637365156,01/01/2005,None,5.75,130964.67,9.0,351.0,351.0,04/2034,35840.0,...,null,null,null,null,null,null,null,None,null,None
1,548637365156,01/01/2006,None,5.75,129125.98,21.0,339.0,338.0,04/2034,35840.0,...,null,null,null,null,null,null,null,None,null,None
2,548637365156,01/01/2007,None,5.75,127061.77,33.0,327.0,325.0,04/2034,35840.0,...,null,null,null,null,null,null,null,None,null,None
3,548637365156,02/01/2005,None,5.75,130820.14,10.0,350.0,350.0,04/2034,35840.0,...,null,null,null,null,null,null,null,None,null,None
4,548637365156,02/01/2006,None,5.75,128972.64,22.0,338.0,337.0,04/2034,35840.0,...,null,null,null,null,null,null,null,None,null,None


In [11]:
to_write_ddf = perf_ddf.repartition(npartitions=16*6)
#to_write_ddf = perf_ddf
#to_write_ddf = to_write_ddf.persist()
to_write_ddf

,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,current_loan_delinquency_status,mod_flag,zero_balance_code,zero_balance_effective_date,last_paid_installment_date,foreclosed_after,disposition_date,foreclosure_costs,prop_preservation_and_repair_costs,asset_recovery_costs,misc_holding_expenses,holding_taxes,net_sale_proceeds,credit_enhancement_proceeds,repurchase_make_whole_proceeds,other_foreclosure_proceeds,non_interest_bearing_upb,principal_forgiveness_upb,repurchase_make_whole_proceeds_flag,foreclosure_principal_write_off_amount,servicing_activity_indicator
npartitions=96,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,float64,float64,float64,float64,float64,object,float64,int32,object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
%%time
compression='snappy'
# compression=None
parquet_dir = os.path.join(data_dir, 'perf-%s.parquet' % str(compression))
print(parquet_dir)
if os.path.exists(parquet_dir): shutil.rmtree(parquet_dir)
to_write_ddf.to_parquet(parquet_dir, compression=compression)

/mnt/isilon1/data/mortgage/perf-snappy.parquet


RuntimeError: RMM error encountered at: /conda/conda-bld/libcudf_1574413108900/work/cpp/src/io/utilities/wrapper_utils.hpp:75: 4 RMM_ERROR_OUT_OF_MEMORY